# Data Cleaning
Here are our solutions for automation of filling missing data.<br>
Main ideology behind this is to train models to recognize patterns like brand, power, etc. in order to classify the correct model (for instance).<br>
We aim to preserve as much data as we can (minimize deletion of records). We are aware that data produced by models may not be credible so might impact the original idea of predicting prices, however it's the path we want to explore.

## Imports and paths

In [1]:
import pandas as pd
import pathlib as pl
import helper_functions as help_me

In [2]:
to_fill_path = pl.Path('datasets', 'filled.csv')

## Data loading

In [3]:
to_fill_df = pd.read_csv(
    filepath_or_buffer=to_fill_path,
    sep=',',
    header=0,
    index_col='ID'
)
to_fill_df.isna().sum()

Cena                   0
Marka_pojazdu       1686
Model_pojazdu       1603
Wersja_pojazdu     16296
Rok_produkcji       1737
Moc_KM              2071
Pojemnosc_cm3       2623
Rodzaj_paliwa       1753
Naped               8492
Skrzynia_biegow     2082
Typ_nadwozia        1699
Liczba_drzwi        2684
dtype: int64

## Create training dataset & testing datasets
In this chapter We are focusing on creating datasets (one training and few testing) to predict targets of chosen column.

In [4]:
training_set = to_fill_df.dropna(
    subset=["Marka_pojazdu", "Model_pojazdu", "Wersja_pojazdu", "Rok_produkcji", "Moc_KM", "Pojemnosc_cm3", "Rodzaj_paliwa", "Naped", "Skrzynia_biegow", "Typ_nadwozia", "Liczba_drzwi"]
)
training_set

,Cena,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Moc_KM,Pojemnosc_cm3,Rodzaj_paliwa,Naped,Skrzynia_biegow,Typ_nadwozia,Liczba_drzwi
ID,,,,,,,,,,,,
1,13900.0,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,170.0,1998.0,Gasoline,Front wheels,Manual,minivan,5.0
2,25900.0,Renault,Megane,1.6 16V 110,2010.0,110.0,1598.0,Gasoline,Front wheels,Manual,station_wagon,5.0
4,5999.0,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,90.0,1560.0,Diesel,Front wheels,Manual,compact,5.0
8,11900.0,Renault,Scenic,1.5 dCi Authentique,2011.0,105.0,1461.0,Diesel,Front wheels,Manual,minivan,5.0
10,38900.0,Audi,A4,2.0 TDI,2011.0,143.0,1968.0,Diesel,Front wheels,Manual,sedan,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
135381,3000.0,Citroën,Xsara,II 1.6i Exclusive,2002.0,110.0,1587.0,Gasoline,Front wheels,Manual,compact,5.0
135385,32500.0,Opel,Insignia,2.8 T V6 Cosmo 4x4,2009.0,260.0,2792.0,Gasoline,4x4 (attached automatically),Automatic,sedan,4.0
135389,2300.0,Peugeot,307,2.0 HDi,2004.0,90.0,1997.0,Diesel,Front wheels,Manual,city_cars,5.0


In [5]:
brand_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Marka_pojazdu'
)

model_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Model_pojazdu'
)

version_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Wersja_pojazdu'
)

year_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Rok_produkcji'
)

power_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Moc_KM'
)

capacity_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Pojemnosc_cm3'
)

fuel_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Rodzaj_paliwa'
)

drive_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Naped'
)

transmission_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Skrzynia_biegow'
)

body_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Typ_nadwozia'
)

door_testing_set = help_me.create_testing_set(
    df=to_fill_df,
    target_column='Liczba_drzwi'
)


In [6]:
filled_df = help_me.auto_test(
    training_set=training_set,
    testing_set=brand_testing_set
)

DEBUG:helper_functions:Target column: Marka_pojazdu, Target type: object
DEBUG:helper_functions:Classification


KeyboardInterrupt: 

In [36]:
merged_df = pd.merge(
    left=to_fill_df,
    right=filled_df,
    on='ID',
    how='left',
    suffixes=('', '_new')
)

merged_df['Model_pojazdu'] = merged_df['Model_pojazdu'].fillna(value=merged_df['Model_pojazdu_new'])
merged_df.drop(columns='Model_pojazdu_new', axis=1).iloc[130:140]

,Cena,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Moc_KM,Pojemnosc_cm3,Rodzaj_paliwa,Naped,Skrzynia_biegow,Typ_nadwozia,Liczba_drzwi
ID,,,,,,,,,,,,
132,35900.0,Ford,Galaxy,2.0 TDCi DPF,Mk2 (2006-2015),140.0,1998.0,Diesel,Front wheels,Automatic,minivan,5.0
133,4918.0,Volkswagen,Golf,NaN,IV (1997-2006),105.0,1598.0,Gasoline,Front wheels,Manual,compact,3.0
134,11900.0,BMW,Seria 1,120d,E81/E82/E87/E88 (2004-2013),177.0,1995.0,Diesel,Rear wheels,Manual,city_cars,3.0
135,79900.0,Ford,Mondeo,2.0 EcoBlue ST-Line Plus,Mk5 (2014-),150.0,1997.0,Diesel,Front wheels,Manual,compact,5.0
136,4899.0,Mitsubishi,Colt,1.5 DID Invite,Z30 (2004-2012),95.0,1493.0,Diesel,Front wheels,Manual,city_cars,3.0
137,43900.0,DS Automobiles,NaN,NaN,NaN,120.0,1600.0,Diesel,NaN,NaN,station_wagon,5.0
138,115000.0,Mazda,6,2.0 SkyEnergy,III (2012-),165.0,1998.0,Gasoline,Front wheels,Automatic,sedan,4.0
139,92500.0,Mazda,6,NaN,NaN,253.0,2488.0,Gasoline,Front wheels,Automatic,sedan,5.0
140,8800.0,Opel,Vectra,1.9 CDTI Cosmo ActiveSelect,C (2002-2008),150.0,1910.0,Diesel,NaN,Manual,station_wagon,5.0


In [37]:
merged_df.isna().sum()

Cena                      0
Marka_pojazdu          3274
Model_pojazdu          2008
Wersja_pojazdu        48038
Generacja_pojazdu     41646
Moc_KM                 3732
Pojemnosc_cm3          4683
Rodzaj_paliwa          3410
Naped                 13044
Skrzynia_biegow        3771
Typ_nadwozia           3358
Liczba_drzwi           4362
Model_pojazdu_new    134096
dtype: int64